광고 제거

In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [18]:
data = pd.read_csv('네이버 뷰 술집.csv', encoding='utf-8-sig')
data

,Unnamed: 0,time,review,comment,like,url
0,0,2021. 1. 11. 21:18,냠냠 영등포 술집 아트몬스터 - 힙한 분위기 펍 수제맥주...,[],14,https://blog.naver.com/blossomnn/222204199186
1,1,2021. 1. 22. 13:26,서울 [을지로술집] 분위기좋은 을지로 와인바 추천 : 십분의일...,[],22,https://blog.naver.com/qls1021/222216086056
2,2,2021. 1. 22. 10:24,서울 북부 [아현동 맛집] 37년 전통의 숨은 고수 - ...,[],33,https://blog.naver.com/nidduguy/222215890461
3,3,2021. 1. 12. 8:10,술 캐주얼한 분위기의 신당동 와인바 부우이BUOY 배달 가능 ...,[],9,https://blog.naver.com/psk2109/222204386338
4,4,2021. 1. 15. 15:12,악마의맛집 코로나가 끝나면꼭! 다녀와야하는 술집 주신당 ...,[],4,https://blog.naver.com/dbrrkq12/222208287335
...,...,...,...,...,...,...
112,112,2021. 1. 12. 21:15,먹거리.맛집.카페 분위기까지 맛있는 힙지로 맛집 줄리아 ...,[],1,https://blog.naver.com/w_bell/222205300774
113,113,2021. 1. 8. 17:10,술과 나 8밸리스 까베르네 소비뇽 : 너무나도 생소한 칠레 가...,[],2,https://blog.naver.com/press_won/222200882025
114,114,2021. 1. 31. 23:33,Da(ng)ily 2020년 1월의 기록 Hodan...,[],[],https://blog.naver.com/lhojoon0817/222226977975
115,115,2021. 1. 29. 8:10,일상. vida diaria 2020년 연말정산 -4분기- 혼...,[],7,https://blog.naver.com/switchonu/222222776892


In [19]:
del data['Unnamed: 0']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   time     117 non-null    object
 1   review   117 non-null    object
 2   comment  117 non-null    object
 3   like     117 non-null    object
 4   url      117 non-null    object
dtypes: object(5)
memory usage: 4.7+ KB


### 광고글 특징
1. 특정 단어 포함한 게시글 : '제공받아', '지원받아','해당업체'
2. (네이버 블로그만의 특징) 게시글 길이가 3000자 이상이면 광고글 상위 노출 조건에 해당하여 광고로 볼 수 있음

1번

In [20]:
# 광고글 특징을 stopwords로 지정하여 해당 글은 삭제할 수 있게 함
# 분석가 판단 하에 추가 광고 문구 확인하여 stopwords에 추가

Stopwords = ['제공받아','제공받았','지원받아','지원받았','해당업체','해시태그','업체로부터',
             '체험후기','무상지급받은','무상지급','카카오톡플러스친구','톡톡하세요','네이버톡톡',
             '네이버 톡톡','무상지원','카카오톡 플러스친구','스토리링크','카카오 채널']


In [21]:
data = data[~data['review'].str.contains('|'.join(Stopwords))]
data = data.reset_index(drop=True)
data

,time,review,comment,like,url
0,2021. 1. 11. 21:18,냠냠 영등포 술집 아트몬스터 - 힙한 분위기 펍 수제맥주...,[],14,https://blog.naver.com/blossomnn/222204199186
1,2021. 1. 22. 13:26,서울 [을지로술집] 분위기좋은 을지로 와인바 추천 : 십분의일...,[],22,https://blog.naver.com/qls1021/222216086056
2,2021. 1. 22. 10:24,서울 북부 [아현동 맛집] 37년 전통의 숨은 고수 - ...,[],33,https://blog.naver.com/nidduguy/222215890461
3,2021. 1. 12. 8:10,술 캐주얼한 분위기의 신당동 와인바 부우이BUOY 배달 가능 ...,[],9,https://blog.naver.com/psk2109/222204386338
4,2021. 1. 15. 15:12,악마의맛집 코로나가 끝나면꼭! 다녀와야하는 술집 주신당 ...,[],4,https://blog.naver.com/dbrrkq12/222208287335
...,...,...,...,...,...
112,2021. 1. 12. 21:15,먹거리.맛집.카페 분위기까지 맛있는 힙지로 맛집 줄리아 ...,[],1,https://blog.naver.com/w_bell/222205300774
113,2021. 1. 8. 17:10,술과 나 8밸리스 까베르네 소비뇽 : 너무나도 생소한 칠레 가...,[],2,https://blog.naver.com/press_won/222200882025
114,2021. 1. 31. 23:33,Da(ng)ily 2020년 1월의 기록 Hodan...,[],[],https://blog.naver.com/lhojoon0817/222226977975
115,2021. 1. 29. 8:10,일상. vida diaria 2020년 연말정산 -4분기- 혼...,[],7,https://blog.naver.com/switchonu/222222776892


2번

In [22]:
ix = data['review'].str.len() > 3000
data = data.loc[~ix]
data = data.reset_index(drop=True)
data

,time,review,comment,like,url
0,2021. 1. 11. 21:18,냠냠 영등포 술집 아트몬스터 - 힙한 분위기 펍 수제맥주...,[],14,https://blog.naver.com/blossomnn/222204199186
1,2021. 1. 22. 13:26,서울 [을지로술집] 분위기좋은 을지로 와인바 추천 : 십분의일...,[],22,https://blog.naver.com/qls1021/222216086056
2,2021. 1. 15. 15:12,악마의맛집 코로나가 끝나면꼭! 다녀와야하는 술집 주신당 ...,[],4,https://blog.naver.com/dbrrkq12/222208287335
3,2021. 1. 18. 11:41,유은타워7th 유은타워7차에 뉴트로 감성 술집 창업 추천드려요...,[],0,https://blog.naver.com/yoobo1234/222211103126
4,2021. 1. 21. 10:00,"맛집후기 신촌 술집 아무(Amu) - 신촌 숨은 맛집, ...",[],35,https://blog.naver.com/gyfhx/222214630007
5,2021. 1. 19. 7:00,핫 플레이스 핫 플레이스<14> 레트로 감성충만 을지로 <소규...,[],47,https://blog.naver.com/dasanpartner/222211460802
6,2021. 1. 22. 1:29,b o b j i b [서울 을지로] 중식당 줄리아 ...,[],5,https://blog.naver.com/traveler_01/222215650757
7,2021. 1. 28. 17:43,"길잡이별 210127_을지로(삼곱식당,보틀러) 두두...",[],5,https://blog.naver.com/hhs_1226/222223200271
8,2021. 1. 16. 15:29,서울에서 생긴 일 구의역맛집 풘샵 신상 술집 서울토...,[],6,https://blog.naver.com/zaizzaiz/222209308954
9,2021. 1. 8. 10:03,맛집 [강남/역삼] 시원한 맥주가 있는 레트로 감성 넘치는 강...,[],27,https://blog.naver.com/lovely_peace/222200485968


In [23]:
# 기본 전처리
import re

def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus


In [24]:
# 블로그 본문, 댓글 합쳐 하나의 document로 만들기

data['doc'] = data['review'] + ' ' + data['comment']
data.head()

,time,review,comment,like,url,doc
0,2021. 1. 11. 21:18,냠냠 영등포 술집 아트몬스터 - 힙한 분위기 펍 수제맥주...,[],14,https://blog.naver.com/blossomnn/222204199186,냠냠 영등포 술집 아트몬스터 - 힙한 분위기 펍 수제맥주...
1,2021. 1. 22. 13:26,서울 [을지로술집] 분위기좋은 을지로 와인바 추천 : 십분의일...,[],22,https://blog.naver.com/qls1021/222216086056,서울 [을지로술집] 분위기좋은 을지로 와인바 추천 : 십분의일...
2,2021. 1. 15. 15:12,악마의맛집 코로나가 끝나면꼭! 다녀와야하는 술집 주신당 ...,[],4,https://blog.naver.com/dbrrkq12/222208287335,악마의맛집 코로나가 끝나면꼭! 다녀와야하는 술집 주신당 ...
3,2021. 1. 18. 11:41,유은타워7th 유은타워7차에 뉴트로 감성 술집 창업 추천드려요...,[],0,https://blog.naver.com/yoobo1234/222211103126,유은타워7th 유은타워7차에 뉴트로 감성 술집 창업 추천드려요...
4,2021. 1. 21. 10:00,"맛집후기 신촌 술집 아무(Amu) - 신촌 숨은 맛집, ...",[],35,https://blog.naver.com/gyfhx/222214630007,"맛집후기 신촌 술집 아무(Amu) - 신촌 숨은 맛집, ..."


In [25]:
data.doc = data.doc.str.replace('[^가-힣]',' ') # 한글만 남기고 특수문자, 숫자, 영어 삭제
data.doc = data.doc.str.replace('\s+',' ')  # white space(빈칸) 삭제

data

C:\Users\user\AppData\Local\Temp\ipykernel_14232\526420279.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace('[^가-힣]',' ') # 한글만 남기고 특수문자, 숫자, 영어 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_14232\526420279.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace('\s+',' ')  # white space(빈칸) 삭제


,time,review,comment,like,url,doc
0,2021. 1. 11. 21:18,냠냠 영등포 술집 아트몬스터 - 힙한 분위기 펍 수제맥주...,[],14,https://blog.naver.com/blossomnn/222204199186,냠냠 영등포 술집 아트몬스터 힙한 분위기 펍 수제맥주 전문점 엔 복사 이웃추가 본...
1,2021. 1. 22. 13:26,서울 [을지로술집] 분위기좋은 을지로 와인바 추천 : 십분의일...,[],22,https://blog.naver.com/qls1021/222216086056,서울 을지로술집 분위기좋은 을지로 와인바 추천 십분의일 디티 복사 이웃추가 본문 ...
2,2021. 1. 15. 15:12,악마의맛집 코로나가 끝나면꼭! 다녀와야하는 술집 주신당 ...,[],4,https://blog.naver.com/dbrrkq12/222208287335,악마의맛집 코로나가 끝나면꼭 다녀와야하는 술집 주신당 루시퍼 복사 이웃추가 본문 ...
3,2021. 1. 18. 11:41,유은타워7th 유은타워7차에 뉴트로 감성 술집 창업 추천드려요...,[],0,https://blog.naver.com/yoobo1234/222211103126,유은타워 유은타워 차에 뉴트로 감성 술집 창업 추천드려요 열심히 사는 대 복사 이...
4,2021. 1. 21. 10:00,"맛집후기 신촌 술집 아무(Amu) - 신촌 숨은 맛집, ...",[],35,https://blog.naver.com/gyfhx/222214630007,맛집후기 신촌 술집 아무 신촌 숨은 맛집 따뜻한 분위기와 맛있는 음식 서이추 환영...
5,2021. 1. 19. 7:00,핫 플레이스 핫 플레이스<14> 레트로 감성충만 을지로 <소규...,[],47,https://blog.naver.com/dasanpartner/222211460802,핫 플레이스 핫 플레이스 레트로 감성충만 을지로 소규모와인바 다산파트너 복사 이웃...
6,2021. 1. 22. 1:29,b o b j i b [서울 을지로] 중식당 줄리아 ...,[],5,https://blog.naver.com/traveler_01/222215650757,서울 을지로 중식당 줄리아 뮥뮥뮹 복사 이웃추가 본문 기타 기능 지도로 보기 전체...
7,2021. 1. 28. 17:43,"길잡이별 210127_을지로(삼곱식당,보틀러) 두두...",[],5,https://blog.naver.com/hhs_1226/222223200271,길잡이별 을지로 삼곱식당 보틀러 두두 복사 이웃추가 본문 기타 기능 지도로 보기 ...
8,2021. 1. 16. 15:29,서울에서 생긴 일 구의역맛집 풘샵 신상 술집 서울토...,[],6,https://blog.naver.com/zaizzaiz/222209308954,서울에서 생긴 일 구의역맛집 풘샵 신상 술집 서울토박이 복사 이웃추가 본문 기타 ...
9,2021. 1. 8. 10:03,맛집 [강남/역삼] 시원한 맥주가 있는 레트로 감성 넘치는 강...,[],27,https://blog.naver.com/lovely_peace/222200485968,맛집 강남 역삼 시원한 맥주가 있는 레트로 감성 넘치는 강남역 술집 랭맥 피뚜 복...


In [26]:
tqdm.pandas()

data['doc'] = data['doc'].progress_apply(lambda x : clean_text(x))

100%|██████████| 53/53 [00:00<00:00, 207.83it/s]


In [28]:
data.to_csv('네이버 뷰 술집(전처리).csv', encoding='utf-8-sig')

### 형태소 분석(pos-tagging)
- 텍스트 분석을 하기 위해서 단어의 형태로 데이터를 처리해 줌
- 단어로 만드는 과정에서 
    1) 분석에 불필요한 불용어 제거 
    2) 단어를 사전형으로 처리(가고있다->가다, 있다)
- 명사, 형용사, 동사, 부사의 품사만 분석 대상으로 선정

In [29]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize
